# Импорты

In [5]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys 
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
from multiprocessing import Pool, cpu_count
from concurrent.futures import ThreadPoolExecutor

In [4]:
import selenium
selenium.__version__

'3.141.0'

# Настройки

In [7]:
# links = pd.read_csv('all_links.csv')

# Поиск информации о товарах

In [8]:
# links['link']

NameError: name 'links' is not defined

In [9]:
driver = webdriver.Edge(executable_path=r'C:\Users\buthy\Downloads\edgedriver_win321\msedgedriver.exe')
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

WebDriverException: Message: 'C:\Users\buthy\Downloads\edgedriver_win321\msedgedriver.exe' executable needs to be in PATH. Please download from http://go.microsoft.com/fwlink/?LinkId=619687


In [39]:
def download_html(links, driver):
    
    df = pd.DataFrame()
    for link in tqdm_notebook(links):
        driver.get(link)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        html = driver.page_source
        df_new = parse_html_to_df(html) 
        df_new['link'] = link
        df = pd.concat([df, df_new])
        df.drop_duplicates()
    return df

def parse_html_to_df(html):
    soup = BeautifulSoup(html,'html.parser') 
    prices_to_parse = soup.find_all(lambda tag: tag.name == 'div' and tag.get('class') == ['product__price'])
    prices = [obj.text.strip() for obj in prices_to_parse]
    title_to_parse = soup.find_all(lambda tag: tag.name == 'p' and tag.get('class') == ['product__title'])
    titles = [obj.text for obj in title_to_parse]
    volum_to_parse = soup.find_all(lambda tag: tag.name == 'p' and tag.get('class') == ['product__volume'])
    volums = [obj.text.strip() for obj in volum_to_parse]
    return pd.DataFrame(np.array([titles, prices, volums]).T, columns=['название', 'цена', 'объем'])

In [40]:
links_to_del = set()
for link in links['link']:
    length = len(link.split('/')[-1])
    link_del = link[:-length-1]
    links_to_del = links_to_del | set([link_del])

In [41]:
links

,link,магазин
0,https://sbermarket.ru/deya,deya
1,https://sbermarket.ru/deya/ovoshchi-i-frukty,deya
2,https://sbermarket.ru/deya/voda-soki-napitki,deya
3,https://sbermarket.ru/deya/moloko-iaitsa,deya
4,https://sbermarket.ru/deya/bakalieia,deya
...,...,...
28054,https://sbermarket.ru/auchansm/tovary-dlia-ofi...,auchansm
28055,https://sbermarket.ru/auchansm/tovary-dlia-ofi...,auchansm
28056,https://sbermarket.ru/deya/moloko-iaitsa/slivo...,deya
28057,https://sbermarket.ru/deya/moloko-iaitsa/slivo...,deya


In [42]:
links = links[~links['link'].isin(links_to_del)].reset_index(drop=True)

In [43]:
array_split = np.array_split(links['link'].values, 12)

In [44]:
def update_df(links):
    driver = driver = webdriver.Edge(executable_path=r'C:\Users\buthy\Downloads\edgedriver_win321\msedgedriver.exe')
    driver.set_script_timeout(600)
    df = download_html(links, driver)
    driver.quit();
    return df

In [ ]:
with ThreadPoolExecutor(12) as executor:
    results = executor.map(update_df, array_split)

C:\Users\buthy\anaconda3\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [16]:
df = pd.DataFrame()
for df_new in results:
    df = pd.concat([df, df_new])
    df.drop_duplicates()

In [20]:
df = df.merge(links, how='left')

In [26]:
all_links = pd.read_csv('shops_25_01_21.csv')

In [33]:
len(all_links['магазин'].unique())

72

In [34]:
all_links_1 = all_links[:60_000] 
all_links_2 = all_links[60_000:120_000] 
all_links_3 = all_links[120_000:180_000] 
all_links_4 = all_links[180_000:] 


In [35]:
all_links_1.to_excel('shops_25_01_21_1.xls', index=False)
all_links_2.to_excel('shops_25_01_21_2.xls', index=False)
all_links_3.to_excel('shops_25_01_21_3.xls', index=False)
all_links_4.to_excel('shops_25_01_21_4.xls', index=False)
# all_links_1.to_excel('shops_25_01_21_1.xls', index=False)

In [96]:
new_links = pd.read_csv('all_links.csv')
new_links.columns = ['link', 'магазин']

In [102]:
links_all_df = pd.concat([new_links, links]) 
links_all_df = links_all_df.drop_duplicates()
links_all_df.to_csv('all_links.csv', index=False)

In [53]:
markets = pd.read_csv('shops.csv')

In [54]:
markets.to_excel('shops.xls', index=False)

In [ ]:
df.to_csv('first_shops.csv', index=False)

In [89]:
df = df.merge(links, on='link', how='left')

In [84]:
df.columns

Index(['название', 'цена', 'объем', 'link'], dtype='object')

In [87]:
links.columns = ['link', 'магазин']

In [49]:
df = df.drop(columns=['index', 0])

In [51]:
df.to_csv('products_ashan.csv', index=False)